Exercise 01:

In [68]:
from sklearn.datasets import load_digits
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection

digits = load_digits()
data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]

print(np.shape(images[0]))
#The images are of size 8x8

#Interpolate using nearest neighbor
img_nearest = images[3]
assert 2 == len (img_nearest.shape)
plt.figure()
plt.gray()
plt.imshow(img_nearest,interpolation = "nearest" ) # also try interpolation =" bicubic "
plt.show()

#Interpolate using bicubic
img_bicubic = images[3]
assert 2 == len (img_bicubic.shape)
plt.figure()
plt.gray()
plt.imshow(img_bicubic,interpolation = "bicubic" ) # also try interpolation =" bicubic "
plt.show()

x_3and9 = []
y_3and9 = []
image_3and9 = []
for i in range(len(data)):
    if target[i] == 3 or target[i] == 9:
        x_3and9.append(data[i])
        y_3and9.append(target[i]) 
        image_3and9.append(images[i])

X_train, X_test, y_train, y_test =\
    model_selection.train_test_split(x_3and9, y_3and9, test_size = 0.4, random_state = 0)


(8, 8)
1.4863013698630136
[ 0.  0.  0.  1. 12. 16. 14.  0.  0.  0.  3. 14. 13. 15. 13.  0.  0.  4.
 16. 15. 13. 16.  4.  0.  0.  3. 16. 16. 16. 16.  3.  0.  0.  0.  7.  7.
 14. 14.  0.  0.  0.  0.  0.  0. 12. 11.  0.  0.  0.  0.  0.  0. 13. 10.
  0.  0.  0.  0.  0.  0. 13. 12.  0.  0.]
[9, 9, 9, 3, 9, 9, 3, 3, 9, 9, 3, 9, 3, 3, 9, 9, 3, 9, 3, 3, 3, 9, 9, 9, 3, 3, 9, 9, 9, 3, 9, 9, 3, 9, 9, 3, 3, 3, 3, 3, 9, 3, 9, 9, 9, 9, 9, 9, 9, 9, 3, 9, 9, 9, 9, 3, 3, 9, 9, 3, 9, 3, 3, 9, 3, 3, 9, 3, 3, 9, 3, 9, 3, 3, 9, 3, 9, 9, 3, 9, 3, 3, 9, 3, 3, 9, 3, 3, 9, 9, 3, 9, 3, 9, 9, 9, 9, 3, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 9, 9, 9, 9, 9, 3, 9, 9, 9, 3, 9, 3, 9, 9, 3, 3, 9, 3, 3, 9, 9, 3, 9, 3, 9, 3, 3, 9, 9, 9, 9, 3, 3, 9, 9, 3, 9, 3, 3, 9, 3, 3, 3, 3, 3, 9, 9, 9, 9, 9, 3, 3, 9, 3, 9, 3, 3, 3, 9, 9, 3, 9, 9, 3, 3, 3, 9, 9, 9, 9, 9, 9, 3, 3, 3, 9, 3, 3, 9, 3, 3, 9, 3, 9, 9, 3, 3, 3, 3, 3, 3, 3, 9, 3, 9, 3, 9, 9, 3, 9, 3, 3, 3, 3, 3, 3, 3, 3]
